In [1]:
import numpy as np
import xarray as xr
import copy 
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

plot_path = '/home/u/u241308/figures/'

In [2]:
ref_hist_min = 1850
ref_hist_max = 1879

In [20]:
lon_eu_max = 30
lon_eu_min = -10
lat_eu_max = 70
lat_eu_min = 35

In [44]:
model_list =  ['CanESM5','MIROC6','ACCESS-ESM1-5']
ens_total_list = [50,50,40]
n_run = 'historical'

In [60]:
for i,model in enumerate(model_list): 
    ens_total = ens_total_list[i]
    
    path = '/work/uo1075/u241308/SMILE/'+model+'/mrso/'
    file = model+'_soil_moisture_'+n_run+'_mm_ens1-%i_GEgrid.nc'%ens_total
    sm = xr.open_dataset(path+file)
    sm = sm.mrso
    sm = sm[(sm.time.dt.month >= 6) & (sm.time.dt.month <= 8)].resample(time='Y').mean()
    
    #select early historical data
    sm = sm[(sm.time.dt.year>=ref_hist_min)&(sm.time.dt.year<=ref_hist_max)]
    
    #Define new longitudes: from 0,360 to -180,180, and select Europe
    sm['lon'] = np.where(sm.lon >180, sm.lon-360,sm.lon)
    ind = np.argsort(sm.lon)
    ind.values
    sm = sm[:,:,:,ind.values]
    #sel lon/lat
    sm = sm[:,:,(sm.lat >= lat_eu_min) & (sm.lat <= lat_eu_max),(sm.lon >= lon_eu_min) & (sm.lon <= lon_eu_max)]

    sm_mean = sm.mean(dim='sfc').mean(dim='time')
    sm_std = sm.std(dim='sfc').mean(dim='time')

    path='/work/uo1075/u241308/data_python_PostDoc/HW_expansion/soil_moisture_smile/'
    file='SM_JJA_Europe_%i-%i_ens_mean_'%(ref_hist_min,ref_hist_max) +model+'.nc'
    sm_mean.to_netcdf(path+file)

    file='SM_JJA_Europe_%i-%i_ens_std_'%(ref_hist_min,ref_hist_max) +model+'.nc'
    sm_std.to_netcdf(path+file)

In [61]:
#for MPI
#first set
path = '/work/uo1075/u241308/grand_ensemble_cmip6/soil_moisture/ens1_30/'
file = 'nGE_soil_moisture_'+n_run+'_mm_ens1-30.nc'
sm1 = xr.open_dataset(path+file)
sm1 = sm1.mrso

sm1 = sm1[(sm1.time.dt.month >= 6) & (sm1.time.dt.month <= 8)].resample(time='Y').mean()

#Define new longitudes: from 0,360 to -180,180, and select Europe
sm1['lon'] = np.where(sm1.lon >180, sm1.lon-360,sm1.lon)
ind = np.argsort(sm1.lon)
ind.values
sm1 = sm1[:,:,:,ind.values]
#sel lon/lat
sm1 = sm1[:,:,(sm1.lat >= lat_eu_min) & (sm1.lat <= lat_eu_max),(sm1.lon >= lon_eu_min) & (sm1.lon <= lon_eu_max)]

#second set
path = '/work/uo1075/u241308/grand_ensemble_cmip6/soil_moisture/ens31_50/'
file = 'nGE_soil_moisture_'+n_run+'_mm_ens31-50.nc'
sm2 = xr.open_dataset(path+file)
sm2 = sm2.mrso

sm2 = sm2[(sm2.time.dt.month >= 6) & (sm2.time.dt.month <= 8)].resample(time='Y').mean()

#Define new longitudes: from 0,360 to -180,180
sm2['lon'] = np.where(sm2.lon >180, sm2.lon-360,sm2.lon)
ind = np.argsort(sm2.lon)
ind.values
sm2 = sm2[:,:,:,ind.values]
#sel lon/lat
sm2 = sm2[:,:,(sm2.lat >= lat_eu_min) & (sm2.lat <= lat_eu_max),(sm2.lon >= lon_eu_min) & (sm2.lon <= lon_eu_max)]

#concatenate two sets
sm = xr.concat((sm1,sm2),dim='sfc')

#select early historical data
sm = sm[(sm.time.dt.year>=ref_hist_min)&(sm.time.dt.year<=ref_hist_max)]

sm_mean = sm.mean(dim='sfc').mean(dim='time')
sm_std = sm.std(dim='sfc').mean(dim='time')

path='/work/uo1075/u241308/data_python_PostDoc/HW_expansion/soil_moisture_smile/'
model='MPI-GE'
file='SM_JJA_Europe_%i-%i_ens_mean_'%(ref_hist_min,ref_hist_max) +model+'.nc'
sm_mean.to_netcdf(path+file)
file='SM_JJA_Europe_%i-%i_ens_std_'%(ref_hist_min,ref_hist_max) +model+'.nc'
sm_std.to_netcdf(path+file)